In [1]:
# Download data from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE75688
!python -V
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE75nnn/GSE75688/suppl/GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt.gz
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE75nnn/GSE75688/suppl/GSE75688_final_sample_information.txt.gz

Python 3.11.11
--2025-04-02 19:31:04--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE75nnn/GSE75688/suppl/GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10799212 (10M) [application/x-gzip]
Saving to: ‘GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt.gz’

GSE75688_GEO_proces 100%[===================>]  10.30M  43.0MB/s    in 0.2s    

2025-04-02 19:31:05 (43.0 MB/s) - ‘GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt.gz’ saved [10799212/10799212]

--2025-04-02 19:31:05--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE75nnn/GSE75688/suppl/GSE75688_final_sample_information.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 130.14.250.13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ft

**1. CARREGAR E PROCESSAR DADOS**

In [2]:
import pandas as pd
import numpy as np
import gzip
from sklearn.decomposition import PCA
from sklearn.metrics import normalized_mutual_info_score
import subprocess

In [3]:
with gzip.open("GSE75688_final_sample_information.txt.gz", "rt") as f:
    sample_info = pd.read_csv(f, sep="\t")
sample_info

,sample,type,index,index2,index3
0,BC01_02,SC,Tumor,Tumor,Tumor
1,BC01_03,SC,Tumor,Tumor,Tumor
2,BC01_04,SC,Tumor,Tumor,Tumor
3,BC01_05,SC,Tumor,Tumor,Tumor
4,BC01_06,SC,Tumor,Tumor,Tumor
...,...,...,...,...,...
523,BC06_58,SC,nonTumor,Immune,Bcell
524,BC06_60,SC,Tumor,Tumor,Tumor
525,BC06_61,SC,nonTumor,Immune,Bcell
526,BC06_74,SC,Tumor,Tumor,Tumor


In [4]:
# Carregar a matriz de expressão gênica
with gzip.open("GSE75688_GEO_processed_Breast_Cancer_raw_TPM_matrix.txt.gz", "rt") as f:
    gene_expression = pd.read_csv(f, sep="\t")
gene_expression

,gene_id,gene_name,gene_type,BC01_Pooled,BC01_Tumor,BC02_Pooled,BC03_Pooled,BC03LN_Pooled,BC04_Pooled,BC05_Pooled,...,BC11_04,BC11_07,BC11_28,BC11_43,BC11_56,BC11_69,BC11_70,BC11_78,BC11_81,BC11_88
0,ENSG00000000003.10,TSPAN6,protein_coding,2.33,1.25,43.96,7.64,9.32,133.37,84.77,...,0.00,47.44,16.11,0.00,6.56,0.00,0.00,0.00,200.82,0.00
1,ENSG00000000005.5,TNMD,protein_coding,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSG00000000419.8,DPM1,protein_coding,60.70,28.44,74.73,41.41,87.69,120.41,70.67,...,198.96,0.00,0.00,20.63,0.00,0.00,0.00,136.07,22.03,204.58
3,ENSG00000000457.9,SCYL3,protein_coding,47.93,4.43,9.89,7.61,7.32,12.42,12.02,...,0.00,10.96,0.00,0.00,0.00,0.27,0.00,0.07,0.19,9.52
4,ENSG00000000460.12,C1orf112,protein_coding,4.79,1.67,10.87,0.92,15.13,17.10,5.69,...,0.00,6.97,9.02,0.00,0.00,0.14,247.69,22.60,11.35,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57910,ERCC-00170,ERCC-00170,ERCC,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
57911,ERCC-00171,ERCC-00171,ERCC,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
57912,SPIKE1,EC2,SPIKE_IN,14940.70,45728.19,10442.91,94250.71,40823.13,24540.58,12613.85,...,166346.51,54443.35,120842.89,135291.06,138048.60,58840.26,65246.34,33465.11,108901.35,57492.01
57913,SPIKE2,EC15,SPIKE_IN,985.82,607.50,362.55,2949.35,1291.13,854.53,399.57,...,6860.84,2136.69,4048.09,4062.59,4147.05,2286.91,2821.49,1652.19,3439.53,1837.41


In [5]:
# Ajustar os nomes das colunas
gene_expression.columns = gene_expression.columns.str.strip()

# Filtrar apenas as colunas que começam com "BC" (mas mantendo "gene_id")
selected_columns = ["gene_id"] + [col for col in gene_expression.columns if col.startswith("BC")]
gene_expression = gene_expression[selected_columns]
gene_expression

,gene_id,BC01_Pooled,BC01_Tumor,BC02_Pooled,BC03_Pooled,BC03LN_Pooled,BC04_Pooled,BC05_Pooled,BC06_Pooled,BC07_Tumor,...,BC11_04,BC11_07,BC11_28,BC11_43,BC11_56,BC11_69,BC11_70,BC11_78,BC11_81,BC11_88
0,ENSG00000000003.10,2.33,1.25,43.96,7.64,9.32,133.37,84.77,48.01,60.29,...,0.00,47.44,16.11,0.00,6.56,0.00,0.00,0.00,200.82,0.00
1,ENSG00000000005.5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.26,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,ENSG00000000419.8,60.70,28.44,74.73,41.41,87.69,120.41,70.67,92.23,32.48,...,198.96,0.00,0.00,20.63,0.00,0.00,0.00,136.07,22.03,204.58
3,ENSG00000000457.9,47.93,4.43,9.89,7.61,7.32,12.42,12.02,7.99,27.70,...,0.00,10.96,0.00,0.00,0.00,0.27,0.00,0.07,0.19,9.52
4,ENSG00000000460.12,4.79,1.67,10.87,0.92,15.13,17.10,5.69,29.20,17.00,...,0.00,6.97,9.02,0.00,0.00,0.14,247.69,22.60,11.35,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57910,ERCC-00170,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
57911,ERCC-00171,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
57912,SPIKE1,14940.70,45728.19,10442.91,94250.71,40823.13,24540.58,12613.85,12759.76,21013.28,...,166346.51,54443.35,120842.89,135291.06,138048.60,58840.26,65246.34,33465.11,108901.35,57492.01
57913,SPIKE2,985.82,607.50,362.55,2949.35,1291.13,854.53,399.57,495.28,288.59,...,6860.84,2136.69,4048.09,4062.59,4147.05,2286.91,2821.49,1652.19,3439.53,1837.41


In [6]:
# Transpor a matriz (mantendo "gene_id" como índice)
gene_expression = gene_expression.set_index("gene_id").T

# Garantir que só estamos pegando amostras que aparecem nos metadados
valid_samples = set(sample_info["sample"])
gene_expression = gene_expression.loc[gene_expression.index.intersection(valid_samples)]

**2. SALVAR OS DADOS SEM REDUÇÃO DIMENSIONAL**

In [7]:
# Salvar a matriz
txt_filename = "genes.data"
np.savetxt(txt_filename, gene_expression, delimiter="\t")
print(f"Arquivo {txt_filename} salvo!")

Arquivo genes.data salvo!


**3. GERAR RESTRIÇÕES (ML/CL)**

In [8]:
ml_constraints = []
cl_constraints = []
sample_to_index = {sample: i for i, sample in enumerate(gene_expression.index)}

# Criar dicionário de categorias
category_dict = sample_info.set_index("sample")["type"].to_dict()

# Criar restrições baseadas no tipo de tumor
samples = list(gene_expression.index)
for i in range(len(samples)):
    for j in range(i + 1, len(samples)):
        if category_dict[samples[i]] == category_dict[samples[j]]:
            ml_constraints.append(f"1 {sample_to_index[samples[i]]} {sample_to_index[samples[j]]}")  # ML = 1
        else:
            cl_constraints.append(f"-1 {sample_to_index[samples[i]]} {sample_to_index[samples[j]]}")  # CL = -1

In [9]:
# Salvar arquivo de restrições no formato correto
with open("genes.constraints", "w") as f:
    for ml in ml_constraints:
        f.write(ml + "\n")
    for cl in cl_constraints:
        f.write(cl + "\n")

print("Arquivo genes.constraints salvo!")

Arquivo genes.constraints salvo!


**4. EXECUTAR COP-KMEANS**

In [10]:
cmd = ["python", "run_ckm.py", "genes.data", "genes.constraints", "3", "--ofile", "output.txt", "--n_rep", "5", "--m_iter", "100", "--tol", "1e-3"]
subprocess.run(cmd)

CompletedProcess(args=['python', 'run_ckm.py', 'genes.data', 'genes.constraints', '3', '--ofile', 'output.txt', '--n_rep', '5', '--m_iter', '100', '--tol', '1e-3'], returncode=0)

**5. CALCULAR NMI**

In [11]:
# Carregar rótulos verdadeiros
true_labels = {}
with gzip.open("GSE75688_final_sample_information.txt.gz", "rt", encoding="utf-8") as f:
    next(f)  # Pular cabeçalho
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 2:
            sample_id, sample_type = parts[0], parts[1]
            true_labels[sample_id] = sample_type

# Carregar os clusters gerados pelo COP-KMeans
clusters = []
with open("output.txt", "r") as f:
    for line in f:
        clusters.append(int(line.strip()))

# Converter rótulos verdadeiros em valores numéricos
unique_labels = list(set(true_labels.values()))
label_to_int = {label: i for i, label in enumerate(unique_labels)}
true_labels_numeric = [label_to_int[true_labels[sample]] for sample in gene_expression.index]

# Calcular o NMI
nmi_score = normalized_mutual_info_score(true_labels_numeric[:len(clusters)], clusters)
print(f"Normalized Mutual Information (NMI): {nmi_score:.4f}")

🔹 Normalized Mutual Information (NMI): 0.0245
